In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys
import tensorflow as tf

from joblib import load

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [ ]:
########################################
########### Prepare Data ###############
########################################
from modules.data_processor import generate_data

path = '/home/Data/Timeseries/Classification/anomaly_new/anomaly_dataset.pickle'
trainX, trainY, valX, valY, testX, testY, classes, seqlen, channel = generate_data(path, create_val=True, verbose=1)

trainLen, valLen, testLen = trainX.shape[0], valX.shape[0], testX.shape[0]

set_name = path.split(os.sep)[-2]
model_path = os.path.join('../../models', set_name)
img_path = os.path.join('../../images', set_name)
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(img_path):
    os.makedirs(img_path)

In [ ]:
# not modularized

def validate_and_adjust_settings(zero, attach, notemp):
    # 0 0 0 invalid
    # 1 0 0 valid
    # 0 1 0 valid
    # 1 1 0 valid
    # 0 0 1 invalid
    # 1 0 1 valid
    # 0 1 1 invalid
    # 1 1 1 valid
    if zero == 0 and attach == 0:
        return 1, attach, notemp
    if attach == 0 and notemp == 0:
        return 1, attach, notemp
    if notemp == 1:
        return 1, attach, notemp
    return zero, attach, notemp

def define_setup(config, zero, attach, notemp):
    s = 'strides_'
    l = 'length_'
    for c in config:
        s += str(c[0]) + '-'
        l += str(c[1]) + '-'
    s = s[:-1] + '_' + l[:-1] +'_zero-'
    s += '1' if zero else '0'
    s += '_attach-'
    s += '1' if attach else '0'
    s += '_notemp-' 
    s += '1' if notemp else '0'
    return s

In [ ]:
########################################
############# Patch Data ###############
########################################
from modules.data_generator import DataGenerator
from modules.patch_generator import get_generator_id_list

# [Stride, Length]
config = [[5,5]]
zero, attach, notemp = True, True, False
zero, attach, notemp = validate_and_adjust_settings(zero, attach, notemp)

params = {'dim': [seqlen, channel], 'batch_size': 1024, 'config': config,
          'zero': zero, 'attach': attach, 'notemp': notemp, 'shuffle': False}

setup = define_setup(config, zero, attach, notemp)
setup_path = os.path.join(model_path, setup)
if not os.path.exists(setup_path):
    os.makedirs(setup_path)
image_path = os.path.join(img_path, setup, 'sota')
if not os.path.exists(image_path):
    os.makedirs(image_path)

# Generators
trainIds = get_generator_id_list(trainLen, seqlen, config)
train_generator = DataGenerator(trainIds, trainX, trainY, **params)
valIds = get_generator_id_list(valLen, seqlen, config)
val_generator = DataGenerator(valIds, valX, valY, **params)
testIds = get_generator_id_list(testLen, seqlen, config)
test_generator = DataGenerator(testIds, testX, testY, **params)

In [ ]:
########################################
############ Train Level 1 #############
########################################
from modules.model import create_model
from modules.model_trainer import train_descriptive

input_shape = trainX.shape[1:]
if attach:
    input_shape = list(input_shape)
    input_shape[-1] +=1
    input_shape = tuple(input_shape)
patch_model_path = os.path.join(setup_path, 'patch_classifier.h5')

patch_model = load_model(patch_model_path)

softmax_trainXp = patch_model.predict(train_generator)[:len(trainIds)]
softmax_valXp = patch_model.predict(val_generator)[:len(valIds)]
softmax_testXp = patch_model.predict(test_generator)[:len(testIds)]


In [ ]:
########################################
########## Prepare Level 2 #############
########################################
from modules.patch_generator import get_data_patch_stats, get_sample_id_list, compute_trivial_preds

train_pps = get_data_patch_stats(trainLen, seqlen, config)[1]
train_sidx = get_sample_id_list(trainLen, train_pps)
val_pps = get_data_patch_stats(valLen, seqlen, config)[1]
val_sidx = get_sample_id_list(valLen, val_pps)
test_pps = get_data_patch_stats(testLen, seqlen, config)[1]
test_sidx = get_sample_id_list(testLen, test_pps)

In [ ]:
########################################
############ Train Blackbox ############
########################################
from modules.model_trainer import train_blackbox
from modules.data_generator import DataGenerator_simple

# Generators
params_simple = {'dim': [seqlen, channel], 'batch_size': 32, 'shuffle': False}
train_generator_simple = DataGenerator_simple(np.arange(trainLen), trainX, trainY, **params_simple)
val_generator_simple = DataGenerator_simple(np.arange(valLen), valX, valY, **params_simple)
test_generator_simple = DataGenerator_simple(np.arange(testLen), testX, testY, **params_simple)

blackbox_model_path = os.path.join(model_path, 'blackbox_classifier.h5')
blackbox_model = load_model(blackbox_model_path)

bm_train_pred = np.argmax(blackbox_model.predict(train_generator_simple), axis=-1)[:trainLen]
bm_val_pred = np.argmax(blackbox_model.predict(val_generator_simple), axis=-1)[:valLen]
bm_test_pred = np.argmax(blackbox_model.predict(test_generator_simple), axis=-1)[:testLen]

In [ ]:
########################################
####### Parameter SOTA comparison ######
########################################
idx = 1
save = True
# lime
num_samples = 5000
num_features = 10
num_slices = 10
# shap
num_random = 5000


In [ ]:
########################################
########### Plot Statistics ############
########################################
from modules.patch_generator import get_all_patch_params, get_data_patch_stats, get_all_patch, get_patch_params_list

npc, pps = get_data_patch_stats(trainLen, seqlen, config)
ids = get_all_patch_params(idx, npc, pps)
samples = get_all_patch(ids, trainX, trainLen, seqlen, config, zero, attach, notemp)

patch_dists = patch_model.predict(samples)
param_list = get_patch_params_list(ids, trainLen, seqlen, config)

plt.xticks([])
plt.yticks([])
plt.plot(trainX[idx])
for i, d in enumerate(patch_dists):
    plt.axvspan(param_list[i,1] , param_list[i,2], color='C1' if d[1] > d[0] else 'C0', alpha=np.max(d)*0.5)

if save:
    name = 'patchx_' + str(idx)
    plt.savefig(os.path.join(image_path, name + '.png'), dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()


In [ ]:
########################################
############ Lime for time #############
########################################
from modules.lime_timeseries import LimeTimeSeriesExplainer

explainer = LimeTimeSeriesExplainer(class_names=[str(i) for i in range(classes)])
exp = explainer.explain_instance(trainX[idx], blackbox_model.predict, num_features=num_features, num_samples=num_samples, num_slices=num_slices, 
                                 replacement_method='total_mean', is_deep=True)
#fig1 = exp.as_pyplot_figure()
#plt.show()

values_per_slice = np.ceil(len(trainX[idx]) / num_slices)
plt.xticks([])
plt.yticks([])
plt.plot(trainX[idx])

for i in range(num_features):
    feature, weight = exp.as_list()[i]
    start = feature * values_per_slice
    end = start + values_per_slice
    color = 'C0' if weight < 0 else 'C1'
    plt.axvspan(start , end, color=color, alpha=abs(weight)*0.5)

if save:
    name = 'lime_' + str(idx)
    plt.savefig(os.path.join(image_path, name + '.png'), dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
########################################
############ Shap for time #############
########################################
import shap

random_ind = np.random.choice(trainX.shape[0], num_random, replace=False)
e = shap.DeepExplainer((blackbox_model.layers[0].input, blackbox_model.layers[-2].output), trainX[random_ind])

shap_val = e.shap_values(np.expand_dims(trainX[idx], axis=0))
shap_abs = np.absolute(shap_val)
sum_0 = np.sum(shap_abs,axis=0)[0]

sum_0_red = np.sum(sum_0, axis=-1) 
sum_0_red /= np.max(sum_0_red)
mi, ma = np.min(trainX[idx]), np.max(trainX[idx])

pred = np.argmax(blackbox_model.predict(np.expand_dims(trainX[idx], axis=0)))

plt.xticks([])
plt.yticks([])
plt.plot(trainX[idx])
for i, d in enumerate(sum_0_red):
    plt.axvspan(i-1 , i+1, color='C' + str(pred), alpha=d*0.5)

if save:
    name = 'shap_' + str(idx)
    plt.savefig(os.path.join(image_path, name + '.png'), dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()